In [1]:
from __future__ import annotations
from typing import Union, List
import metadsl
import metadsl_rewrite
from metadsl_core.strategies import register_core

In [2]:
class Projection(metadsl.Expression):
    """Projection expression."""
    
    
class Column(metadsl.Expression):
    """Column expression"""

    
class Table(metadsl.Expression):
    """Table base expression class."""
    @metadsl.expression
    def __getitem__(
        self, key: Union[str, List[str]]
    ) -> Union[Projection, Column]:
        """Get item from Table."""

In [3]:
@metadsl.expression
def table(data: dict) -> Table:
    """Create a table expression."""
    
@metadsl.expression
def projection(table: Table, keys: list) -> Projection:
    """Create a projection expression."""
    
@metadsl.expression
def column(table: Table, key: str) -> Column:
    """Create a column expression."""

In [4]:
expr = table({'a': [1, 2, 3]})

In [5]:
expr['a']

metadsl.expressions.PlaceholderExpression[typing.Union[__main__.Projection, __main__.Column]](Table.__getitem__, [__main__.Table(table, [{'a': [1, 2, 3]}], {}), 'a'], {})

In [6]:
expr[['a']]

metadsl.expressions.PlaceholderExpression[typing.Union[__main__.Projection, __main__.Column]](Table.__getitem__, [__main__.Table(table, [{'a': [1, 2, 3]}], {}), ['a']], {})

In [7]:
def rewrite(fn):
    register_core(metadsl_rewrite.rule(fn))

    
def _getitem_str(t: Table, k: list):
    return (
        # expression to match agains
        t[k],
        # expression it is replaced with
        lambda: projection(t, k)
    )


def _getitem_list(t: Table, k: str):
    return (
        # expression to match agains
        t[k],
        # expression it is replaced with
        lambda: column(t, k)
    )


rewrite(_getitem_str)
rewrite(_getitem_list)

In [8]:
metadsl_rewrite.execute(expr['a'])

__main__.Column(column, [__main__.Table(table, [{'a': [1, 2, 3]}], {}), 'a'], {})

In [9]:
metadsl_rewrite.execute(expr[['a']])

__main__.Projection(projection, [__main__.Table(table, [{'a': [1, 2, 3]}], {}), ['a']], {})

In [10]:
expr[['a']]

metadsl.expressions.PlaceholderExpression[typing.Union[__main__.Projection, __main__.Column]](Table.__getitem__, [__main__.Table(table, [{'a': [1, 2, 3]}], {}), ['a']], {})